In [78]:
import pandas as pd
import json
import haversine
import google_analytics_reporting_api as ga
import numpy as np

analytics = ga.initialize_analyticsreporting('/home/jian/Projects/Saatva/GA/GA_api_jubapluscc@gmail.com.json')

Start_Date='2016-11-01'
End_Date='2018-03-01'

GA_Saatva_Geo=ga.get_report_over_10000_rows(analytics=analytics, VIEW_ID='33605697',start_date=Start_Date, end_date=End_Date,
                                            ga_metrics=['ga:sessions','ga:bounceRate','ga:transactions','ga:transactionsPerSession'],
                                            ga_dimensions=['ga:latitude','ga:longitude'])
GA_Saatva_Geo=GA_Saatva_Geo[['ga:latitude','ga:longitude','ga:sessions','ga:bounceRate']]
GA_Saatva_Geo_Zip=GA_Saatva_Geo[['ga:latitude','ga:longitude']]
GA_Saatva_Geo_Zip['ga:latitude']=GA_Saatva_Geo_Zip['ga:latitude'].astype(float)
GA_Saatva_Geo_Zip['ga:longitude']=GA_Saatva_Geo_Zip['ga:longitude'].astype(float)
GA_Saatva_Geo_Zip['closet_zip']=np.nan
GA_Saatva_Geo_Zip['miles']=np.nan

zip_lat_long_josn=json.load(open("/home/jian/Docs/Geo mapping/center_of_rentrak_zip.json"))

Key_list=list(zip_lat_long_josn.keys())

df_zip_center=pd.DataFrame(zip_lat_long_josn).T
df_zip_center.reset_index(inplace=True)
df_zip_center.columns=['zip','latitude','longitude']

max_lat=max(df_zip_center['latitude'])
min_lat=min(df_zip_center['latitude'])

max_long=max(df_zip_center['longitude'])
min_long=min(df_zip_center['longitude'])

GA_Saatva_Geo_Zip=GA_Saatva_Geo_Zip[(GA_Saatva_Geo_Zip['ga:latitude']<max_lat+5) & 
                                   (GA_Saatva_Geo_Zip['ga:latitude']>min_lat-5) & 
                                   (GA_Saatva_Geo_Zip['ga:longitude']<max_long+5) &  
                                   (GA_Saatva_Geo_Zip['ga:longitude']>min_long-5)]
GA_Saatva_Geo_Zip.reset_index(inplace=True)
del GA_Saatva_Geo_Zip['index']

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-vers

In [79]:
GA_Saatva_Geo_Zip=GA_Saatva_Geo_Zip.iloc[0:3,:]
for i in range(len(GA_Saatva_Geo_Zip)):
    distance_list=[]
    for j in range(len(Key_list)):        
        distance=haversine.haversine((GA_Saatva_Geo_Zip.iloc[i,0:2]),(zip_lat_long_josn[Key_list[j]]),miles=True)
        distance_list.append(distance)
    dis_min=min(distance_list)
    index=np.argmin(distance_list)
    GA_Saatva_Geo_Zip['closet_zip'][i]=Key_list[index]
    GA_Saatva_Geo_Zip['miles'][i]=dis_min
    

In [80]:
GA_Saatva_Geo_Zip

,ga:latitude,ga:longitude,closet_zip,miles
0,14.6349,-90.5069,33041.0,888.615936
1,14.9056,-92.2634,78521.0,831.379114
2,15.8720,-97.0767,78521.0,694.521187


In [58]:
GA_Saatva_Geo_Zip.to_csv("/home/jian/Projects/Saatva/ga_lat_long.csv",index=False)